# Runnable

## 订阅发布机制

使用 pub/sub 机制实现动运行时的变量传递。

### 在两个实例之间订阅

In [1]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        self._last_input = prompt
        self._last_output = f'你说了 {prompt}'
        print(prompt)

In [3]:
a = A()
a.call("hi")
a.exported_vars

hi


{'last_input': 'hi', 'last_output': '你说了 hi'}

In [4]:
b = A()
b.bind_runnables(a)
b.imported_vars

{'last_input': 'hi', 'last_output': '你说了 hi'}

## 使用映射规则

In [4]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        pass

class B(Runnable):
    def call(self):
        pass

a1 = A()
a2 = A()
b = B()
b.bind_runnables(a1, binding_map={"a1": "last_output"})
b.bind_runnables(a2, binding_map={"a2": "last_output"})
a1._last_output = "hi"
a2._last_output = "我是黑牛"

print("a1 exported:", a1.exported_vars)
print("a2 exported:", a2.exported_vars)
print("b imported:", b.imported_vars)

a1 exported: {'last_input': None, 'last_output': 'hi'}
a2 exported: {'last_input': None, 'last_output': '我是黑牛'}
b imported: {'last_input': None, 'a1': 'hi', 'a2': '我是黑牛'}


## 在容器实例内订阅

In [5]:
from illufly.types import Runnable

# 用于被嵌入
class A(Runnable):
    def call(self, prompt: str):
        self._last_input = prompt
        self._last_output = f'{self.imported_vars["last_input"]} {prompt}'

class B(Runnable):
    def __init__(self, obj: Runnable):
        super().__init__()
        obj.bind_runnables(self)
        self.obj = obj

    def call(self, instruction):
        self._last_input = instruction
        self.obj.call("你好")
        print(self.obj.last_output)
a = A()
b = B(a)
b.call("那小子在说 >> ")

那小子在说 >>  你好


In [11]:
b.exported_vars

{'last_input': '那小子在说 >> ', 'last_output': None}

In [12]:
a.exported_vars

{'last_input': '你好', 'last_output': '那小子在说 >>  你好'}